In [1]:

import os, copy, numpy as np, pandas as pd, torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


file_path      = r""
new_data_path  = r""


feature_cols = [
    "Feature1", "Feature2", "Feature3", "Price", "Feature5",
    "Feature6", "Feature7", "Feature8", "Feature9", "Feature10", "Feature11"
]
target_col   = "Demand"
assert "Price" in feature_cols, "feature_cols must include 'Price'！"


normalize_y     = True
lambda_phys     = 1.0          
lambda_data     = 5
population_size = 50            
generations     = 5            
nadam_lr0       = 0.01         
nadam_betas     = (0.9, 0.999)
lbfgs_lr        = 0.01         
lbfgs_max_iter  = 20           
gru_hidden      = 32           
torch.manual_seed(42); np.random.seed(42)


def maybe_make_dummy(path: str, rows: int = 120):
    """dummy if no real data"""
    if os.path.exists(path):
        return
    print(f"[Info] {path} No real data")
    data = {c: np.random.rand(rows)*100 for c in feature_cols}
    data[target_col] = 1000 - 5*data["Price"] + 0.3*data["Feature1"] \
                       -0.2*data["Feature2"] + np.random.randn(rows)*25
    pd.DataFrame(data).to_excel(path, index=False)

maybe_make_dummy(file_path)
df = pd.read_excel(file_path)

X = df[feature_cols].astype("float32").values
y = df[target_col].astype("float32").values
N, d = X.shape
price_idx = feature_cols.index("Price")


scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)

if normalize_y:
    scaler_y = StandardScaler().fit(y.reshape(-1,1))
    y_scaled = scaler_y.transform(y.reshape(-1,1)).reshape(-1)
else:
    y_scaled = y

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32).view(-1, 1)


class GRUPINN(nn.Module):
    """
    Input: (batch, feat)  => unsqueeze(1) → GRU(seq=1) → fc(1)
    """
    def __init__(self, input_dim: int, hidden: int = 64):
        super().__init__()
        self.gru = nn.GRU(input_size=input_dim,
                          hidden_size=hidden,
                          batch_first=True)
        self.fc  = nn.Linear(hidden, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (batch, feat)  →  (batch, 1, feat)
        out, _ = self.gru(x.unsqueeze(1))
        return self.fc(out[:, -1, :])      

def create_model():
    return GRUPINN(d, gru_hidden)


def calc_phys_loss(preds, inputs):
    """∂pred/∂Price  ≤ 0"""
    grads = torch.autograd.grad(preds, inputs,
                                grad_outputs=torch.ones_like(preds),
                                create_graph=True)[0]
    price_grads = grads[:, price_idx]          # (batch,)
    return torch.mean(torch.clamp(price_grads, min=0.0)**2)

def train_one(model, X_train, y_train, epochs: int, lr: float):
    model.train()
    opt_nadam = optim.NAdam(model.parameters(), lr=lr, betas=nadam_betas)

    for _ in range(epochs):
        opt_nadam.zero_grad()
        preds = model(X_train)
        data_loss = torch.mean((preds - y_train)**2)

        X_req = X_train.detach().clone().requires_grad_(True)
        phys_loss = calc_phys_loss(model(X_req), X_req)

        loss = lambda_data*data_loss + lambda_phys*phys_loss
        loss.backward()
        opt_nadam.step()

 
    opt_lbfgs = optim.LBFGS(model.parameters(),
                            lr=lbfgs_lr,
                            max_iter=lbfgs_max_iter)

    def closure():
        opt_lbfgs.zero_grad()
        preds = model(X_train)
        data_l = torch.mean((preds - y_train)**2)
        X_req2 = X_train.detach().clone().requires_grad_(True)
        phys_l = calc_phys_loss(model(X_req2), X_req2)
        total  = lambda_data*data_l + lambda_phys*phys_l
        total.backward()
        return total

    opt_lbfgs.step(closure)

def total_loss(model, X_eval, y_eval):
    with torch.no_grad():
        data_l = torch.mean((model(X_eval) - y_eval)**2)
    X_req = X_eval.detach().clone().requires_grad_(True)
    phys_l = calc_phys_loss(model(X_req), X_req).detach()
    return (lambda_data*data_l + lambda_phys*phys_l).item()


population = [
    {"model": create_model(),
     "lr":    nadam_lr0 * (0.8 + 0.4*np.random.rand())}
    for _ in range(population_size)
]

for gen in range(1, generations+1):
    print(f"\n=== PBT Generation {gen}/{generations} ===")
    losses = []
    for i, indiv in enumerate(population, 1):
        train_one(indiv["model"], X_tensor, y_tensor,
                  epochs=30, lr=indiv["lr"])
        loss_i = total_loss(indiv["model"], X_tensor, y_tensor)
        losses.append(loss_i)
        print(f"  ▸ indiv{i:02d} | lr={indiv['lr']:.5f} | loss={loss_i:.6f}")

    best_idx, worst_idx = int(np.argmin(losses)), int(np.argmax(losses))
    if best_idx != worst_idx:
        
        population[worst_idx]["model"].load_state_dict(
            copy.deepcopy(population[best_idx]["model"].state_dict()))
        new_lr = population[best_idx]["lr"] * (0.8 + 0.4*np.random.rand())
        population[worst_idx]["lr"] = new_lr
        print(f"  ↻  Clone best({best_idx+1}) → worst({worst_idx+1}), "
              f"mut lr → {new_lr:.5f}")


best_idx = int(np.argmin([total_loss(p["model"], X_tensor, y_tensor)
                          for p in population]))
best_model = population[best_idx]["model"]
torch.save(best_model.state_dict(), "best_demand_model.pth")
print(f"\n✔ Training finished. Best indiv = {best_idx+1}  "
      f"→ saved to best_demand_model Crude Oil.pth")


best_model.eval()
with torch.no_grad():
    y_pred_train_std = best_model(X_tensor)

y_pred_train = (scaler_y.inverse_transform(y_pred_train_std.numpy())
                if normalize_y else y_pred_train_std.numpy())

df["Predicted"] = y_pred_train
df["Error"]     = df[target_col] - df["Predicted"]
df.to_excel("train_with_predictions Crude Oil.xlsx", index=False)
print("saved as train_with_predictions.xlsx")


maybe_make_dummy(new_data_path, rows=10)
new_df = pd.read_excel(new_data_path)

X_new_std = scaler_X.transform(new_df[feature_cols].astype("float32"))
X_new_tensor = torch.tensor(X_new_std, dtype=torch.float32)

with torch.no_grad():
    y_pred_new_std = best_model(X_new_tensor)
y_pred_new = (scaler_y.inverse_transform(y_pred_new_std.numpy())
              if normalize_y else y_pred_new_std.numpy())
new_df["PredictedDemand"] = y_pred_new
new_df.to_excel("test_with_predictions.xlsx", index=False)
print("saved as test_with_predictions.xlsx")



plt.rcParams["figure.figsize"] = (11,5)
plt.figure()
plt.plot(df.index, df[target_col], label="Train-Actual", marker="o")
plt.plot(df.index, df["Predicted"], label="Train-Pred",  marker="x")
plt.title("Training – Actual vs Predicted"); plt.legend(); plt.tight_layout(); plt.show()

plt.figure()
if target_col in new_df.columns:
    plt.plot(new_df.index, new_df[target_col], label="Test-Actual", marker="o")
plt.plot(new_df.index, new_df["PredictedDemand"], label="Test-Pred", marker="x")
plt.title("Test / Prediction – Actual vs Predicted"); plt.legend(); plt.tight_layout(); plt.show()


[Info]  No real data


ValueError: No engine for filetype: ''